## BERT

(Bidirectional Encoder Representations from Transformers)

입력 시퀀스를 양측에서 처리하여 이전과 이후의 단어를 모두 참조하며 단어의 의미를 파악.

기존 모델들보다 정확하게 문맥을 파악하고 다양한 자연어 처리 작업에서 높은 성능을 보임.

대규모 데이터를 통해 사전 학습되어 있으므로 전이 학습에 주로 활용. 

BERT는 일부나 전체를 다른 작업에서 재사용하여 적은 데이터로도 높은 정확도를 달성. 

### Masked Langauge Modeling, MLM

입력 문장에서 임의로 일부 단어를 마스킹, 해당 단어를 예측.

### Next Sentence Prediction, NSP

두 문장이 주어졌을 때, 두 번째 문장이 첫 번째 문장의 다음에 오는 문장인지의 여부 판단.

BERT 모델의 토큰

CLS / SEP / MASK


CLS:

입력 문장의 시작. 해당 토큰을 사용해 문장 분류 작업을 위한 정보를 제공.

모델은 문장이 어떤 유형의 것인지 미리 파악 가능하게 됨. 


SEP:

입력 문장 내에서 문장의 구분을 위해 사용. 문장 분률 작업에서 복수 문장을 입력으로 받을 때, SEP 토큰을 통해 문장을 구분.

MASK:

입력 문장 내에서 임의로 선택된 단어를 가리키는 토큰. 주어진 문장에서 단어를 가린 후 모델의 학습과 예측에 활용. 

In [1]:
import numpy as np
import pandas as pd
from Korpora import Korpora

In [2]:
corpus = Korpora.load('nsmc')
df = pd.DataFrame(corpus.test).sample(20000, random_state=42)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\dohyeong\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\

In [3]:
train, valid, test = np.split(
    df.sample(frac = 1, random_state = 42), [int(0.6*len(df)), int(0.8*len(df))]
)

C:\Users\dohyeong\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [4]:
print(train.head(5).to_markdown())

|       | text                                                     |   label |
|------:|:---------------------------------------------------------|--------:|
| 26891 | 역시 코믹액션은 성룡, 홍금보, 원표 삼인방이 최고지!!     |       1 |
| 25024 | 점수 후하게 줘야것네 별 반개~                            |       0 |
| 11666 | 오랜만에 느낄수 있는 [감독] 구타욕구.                    |       0 |
| 40303 | 본지는 좀 됬지만 극장서 돈주고 본게 아직까지 아까운 영화 |       0 |
| 18010 | 징키스칸이란 소재를 가지고 이것밖에 못만드냐             |       0 |


In [6]:
import sys
sys.path

['C:\\Users\\dohyeong\\Jupyter Python\\DL',
 '',
 'c:\\users\\dohyeong\\anaconda3\\lib\\site-packages',
 'C:\\Users\\dohyeong\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip',
 'C:\\Users\\dohyeong\\AppData\\Local\\Programs\\Python\\Python311\\DLLs',
 'C:\\Users\\dohyeong\\AppData\\Local\\Programs\\Python\\Python311\\Lib',
 'C:\\Users\\dohyeong\\AppData\\Local\\Programs\\Python\\Python311',
 'C:\\Users\\dohyeong\\AppData\\Roaming\\Python\\Python311\\site-packages',
 'C:\\Users\\dohyeong\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32',
 'C:\\Users\\dohyeong\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib',
 'C:\\Users\\dohyeong\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin',
 'C:\\Users\\dohyeong\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages']

In [10]:
sys.path.append('C:/Users/dohyeong/miniconda3/Lib/site-packages/')

In [11]:
import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler

In [12]:
def make_dataset(data, tokenizer, device):
    tokenized = tokenizer(
        text = data.text.tolist(),
        padding= 'longest',
        truncation = True,
        return_tensors = 'pt'
    )
    input_ids = tokenizedd['input_ids'].to(device)
    attention_mask = tokenized['attention_mask'].to(device)
    labels = torch.tensor(data.label.values, dtype=torch.long).to(device)
    return TensorDataset(input_ids, attention_mask, labels)

In [13]:
def get_dataloader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    dataloader = DataLoader(dataset, sampler = data_sampler, batch_size = batch_size)
    return dataloader

In [15]:
epochs = 5
batch_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path='bert-base-multilingual-cased',
    do_lower_case = False
)

C:\Users\dohyeong\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dohyeong\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [16]:
device

'cpu'

In [17]:
torch.cuda.is_available()

False